# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [34]:
# your code here
cust_amo = orders.groupby('CustomerID')['amount_spent'].agg('sum')
cust_amo.sort_values(ascending=False)
agg_exp = pd.DataFrame(cust_amo).reset_index()
agg_exp
agg_exp.sort_values(by='amount_spent', ascending=False)


,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06
...,...,...
4099,17956,12.75
3015,16454,6.90
1794,14792,6.20
3218,16738,3.75


In [38]:
Lables = ['Preferred','VIP']
bins = pd.qcut(agg_exp['amount_spent'],[0.75,0.95,1],labels=Lables)
print(bins.value_counts())

Preferred    868
VIP          217
Name: amount_spent, dtype: int64


In [39]:
orders['cust_bin'] = bins

In [40]:
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,cust_bin
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,NaN
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,NaN
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,NaN
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,NaN


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [52]:
orders_vip = orders[orders['cust_bin'] == 'VIP']
orders_vip

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,cust_bin
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
10,10,536367,22745,2010,12,3,8,poppy's playhouse bedroom,6,2010-12-01 08:34:00,2.10,13047,United Kingdom,12.60,VIP
12,12,536367,22749,2010,12,3,8,feltcraft princess charlotte doll,8,2010-12-01 08:34:00,3.75,13047,United Kingdom,30.00,VIP
50,50,536373,71053,2010,12,3,9,white metal lantern,6,2010-12-01 09:02:00,3.39,17850,United Kingdom,20.34,VIP
55,55,536373,21071,2010,12,3,9,vintage billboard drink me mug,6,2010-12-01 09:02:00,1.06,17850,United Kingdom,6.36,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4207,6366,536890,17091J,2010,12,5,11,vanilla incense in tin,72,2010-12-03 11:48:00,0.85,14156,EIRE,61.20,VIP
4229,6400,536943,84879,2010,12,5,12,assorted colour bird ornament,8,2010-12-03 12:11:00,1.69,17884,United Kingdom,13.52,VIP
4253,6424,536944,20725,2010,12,5,12,lunch bag red retrospot,70,2010-12-03 12:20:00,1.65,12557,Spain,115.50,VIP
4292,6463,536945,21828,2010,12,5,12,eight piece snake set,1,2010-12-03 12:24:00,1.25,14083,United Kingdom,1.25,VIP


In [53]:
# your code here
orders_vip.groupby(['Country','cust_bin']).agg({'CustomerID':'count'}).sort_values(by='CustomerID', ascending=False)

,,CustomerID
Country,cust_bin,
United Kingdom,VIP,204
Norway,VIP,9
EIRE,VIP,2
Australia,VIP,1
Spain,VIP,1
Australia,Preferred,0
EIRE,Preferred,0
Norway,Preferred,0
Spain,Preferred,0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [54]:
# your code here
orders.groupby(['Country','cust_bin']).agg({'CustomerID':'count'}).sort_values(by='CustomerID', ascending=False)

CustomerID
Country        cust_bin             
United Kingdom Preferred         831
               VIP               204
Norway         VIP                 9
EIRE           Preferred           8
France         Preferred           8
...                              ...
Iceland        VIP                 0
Israel         Preferred           0
               VIP                 0
Italy          Preferred           0
Unspecified    VIP                 0

[74 rows x 1 columns]